# 使用 PyTorch 逐行实现 Transformer

来自b站up主deep_thoughts 合集【PyTorch源码教程与前沿人工智能算法复现讲解】：P_18 到 P_21

P_18 深入刨析 PyTorch 中的 Transformer API 源码：
    
https://www.bilibili.com/video/BV1o44y1Y7cp/?spm_id_from=333.788&vd_source=18e91d849da09d846f771c89a366ed40

P_19 Transformer Encoder 原理精讲及其 PyTorch 逐行实现：

https://www.bilibili.com/video/BV1cP4y1V7GF/?spm_id_from=333.788&vd_source=18e91d849da09d846f771c89a366ed40

P_20 Transformer 模型 Decoder 原理精讲及其 PyTorch 逐行实现：

https://www.bilibili.com/video/BV1Qg411N74v/?spm_id_from=333.788&vd_source=18e91d849da09d846f771c89a366ed40

P_21 Transformer Masked loss 原理精讲及其 PyTorch 逐行实现：

https://www.bilibili.com/video/BV1dh411s7FW/?spm_id_from=333.788&vd_source=18e91d849da09d846f771c89a366ed40

In [15]:
```python
import torch  # 导入PyTorch库，用于构建神经网络和张量操作
import numpy as np  # 导入NumPy库，用于数值计算，虽然在本代码中未直接使用，但可能用于辅助计算
import torch.nn as nn  # 导入PyTorch的神经网络模块，用于定义嵌入层等
import torch.nn.functional as F  # 导入PyTorch的函数模块，用于激活函数、softmax等操作

# 关于word embedding，以序列建模为例
# 考虑source sentence 和 target sentence
# 以下代码演示了Transformer模型中词嵌入（word embedding）的构建过程，以序列建模为例。
# 我们将处理源句子（source sentence）和目标句子（target sentence），这些句子将被转换为词表索引形式，并进行填充（padding）以处理不同长度的序列。

batch_size = 2  # 定义批次大小（batch size），这里设置为2，表示同时处理2个样本

# 单词表大小
max_num_src_words = 8  # 定义源句子词表的最大词汇数量（不包括padding），这里设置为8
max_num_tgt_words = 8  # 定义目标句子词表的最大词汇数量（不包括padding），这里设置为8

model_dim = 8  # 定义模型的嵌入维度（model dimension），即每个词向量的维度，这里设置为8

# 序列的最大长度
max_src_seq_len = 5  # 定义源序列的最大长度，这里设置为5，用于填充padding
max_tgt_seq_len = 5  # 定义目标序列的最大长度，这里设置为5，用于填充padding
max_position_len = 5  # 定义位置编码的最大位置长度，这里设置为5，与序列最大长度相关

#src_len = torch.randint(2, 5, (batch_size,))  # 注释掉的代码：随机生成源序列的实际长度，范围在2到4之间
#tgt_len = torch.randint(2, 5, (batch_size,))  # 注释掉的代码：随机生成目标序列的实际长度，范围在2到4之间
src_len = torch.Tensor([2, 4]).to(torch.int32)  # 定义源序列的实际长度张量，这里手动设置为[2, 4]，表示第一个样本长度2，第二个长度4，并转换为int32类型
tgt_len = torch.Tensor([4, 3]).to(torch.int32)  # 定义目标序列的实际长度张量，这里手动设置为[4, 3]，表示第一个样本长度4，第二个长度3，并转换为int32类型

# 单词索引构成源句子和目标句子， 构建batch， 并且做了padding， 默认值为0
# 构建源序列（src_seq）：对于每个样本，根据其实际长度生成随机词索引（1到max_num_src_words），然后进行padding到最大长度，使用0填充右侧
src_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1, max_num_src_words, (L,)),(0, max(src_len)-L)), 0) for L in src_len])
# 构建目标序列（tgt_seq）：类似源序列，根据实际长度生成随机词索引（1到max_num_tgt_words），padding到最大长度，使用0填充右侧
tgt_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1, max_num_tgt_words, (L,)),(0, max(tgt_len)-L)), 0) for L in tgt_len])

# 构造word embedding
# 创建源词嵌入表（embedding table）：一个嵌入层，将词索引映射到model_dim维向量，词表大小为max_num_src_words+1（包括padding的0）
src_embedding_table = nn.Embedding(max_num_src_words+1, model_dim)
# 创建目标词嵌入表：类似源嵌入表，词表大小为max_num_tgt_words+1
tgt_embedding_table = nn.Embedding(max_num_tgt_words+1, model_dim)
# 获取源序列的词嵌入：将src_seq输入嵌入表，得到形状为[batch_size, max_src_seq_len, model_dim]的张量
src_embedding = src_embedding_table(src_seq)
# 获取目标序列的词嵌入：类似，得到形状为[batch_size, max_tgt_seq_len, model_dim]的张量
tgt_embedding = tgt_embedding_table(tgt_seq)

# 构造position embedding
# 创建位置矩阵（pos_mat）：一个列向量，包含从0到max_position_len-1的位置索引，形状为[max_position_len, 1]
pos_mat = torch.arange(max_position_len).reshape((-1, 1))
# 创建指数矩阵（i_mat）：用于计算正弦/余弦的位置编码，公式为10000^(2i/d_model)，这里i从0到model_dim/2-1，形状为[1, model_dim/2]
i_mat = torch.pow(10000, torch.arange(0, 8, 2).reshape((1, -1))/model_dim)
# 初始化位置嵌入表（pe_embedding_table）：一个零矩阵，形状为[max_position_len, model_dim]
pe_embedding_table = torch.zeros(max_position_len, model_dim)
# 计算偶数维度的位置编码：使用sin(pos / 10000^(2i/d_model))
pe_embedding_table[:, 0::2] = torch.sin(pos_mat / i_mat)
# 计算奇数维度的位置编码：使用cos(pos / 10000^(2i/d_model))
pe_embedding_table[:, 1::2] = torch.cos(pos_mat / i_mat)

# 创建位置嵌入层：一个嵌入层，将位置索引映射到位置编码向量，不需要训练（requires_grad=False）
pe_embedding = nn.Embedding(max_position_len, model_dim)
# 将预计算的位置编码表设置为嵌入层的权重，并设置为不可训练参数
pe_embedding.weight = nn.Parameter(pe_embedding_table, requires_grad=False)

# 构建源位置序列（src_pos）：对于每个样本，生成从0到实际长度-1的位置索引，然后扩展到batch_size，形状为[batch_size, max_src_seq_len]
src_pos = torch.cat([torch.unsqueeze(torch.arange(max(src_len)),0) for _ in src_len]).to(torch.int32)
# 构建目标位置序列（tgt_pos）：类似源位置，形状为[batch_size, max_tgt_seq_len]
tgt_pos = torch.cat([torch.unsqueeze(torch.arange(max(tgt_len)),0) for _ in tgt_len]).to(torch.int32)

# 获取源位置编码：将src_pos输入位置嵌入层，得到形状为[batch_size, max_src_seq_len, model_dim]
src_pe_embedding = pe_embedding(src_pos)
# 获取目标位置编码：类似，得到形状为[batch_size, max_tgt_seq_len, model_dim]
tgt_pe_embedding = pe_embedding(tgt_pos)

# 构造encoder的self-attention mask
# mask的shape：[batch_size, max_src_len, max_src_len],值为1或-inf
# 构建有效编码器位置（valid_encoder_pos）：对于每个样本，创建长度为实际长度的1向量，padding到max_src_seq_len，使用0填充，形状为[batch_size, max_src_seq_len, 1]
valid_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(src_len)-L)),0) for L in src_len]), 2)
# 计算有效位置矩阵：通过批次矩阵乘法（bmm）得到[batch_size, max_src_seq_len, max_src_seq_len]，表示位置i和j是否都有效（非padding）
valid_encoder_pos_matrix = torch.bmm(valid_encoder_pos, valid_encoder_pos.transpose(1, 2))
# 计算无效位置矩阵：1 - valid_encoder_pos_matrix，值为0或1
invalid_encoder_pos_matrix = 1-valid_encoder_pos_matrix
# 转换为布尔掩码：用于后续的masked_fill操作，True表示无效位置（padding）
mask_encoder_self_attention = invalid_encoder_pos_matrix.to(torch.bool)

# 示例分数张量：随机生成[batch_size, max_src_seq_len, max_src_seq_len]的张量，模拟注意力分数
score = torch.randn(batch_size, max(src_len), max(src_len))
# 应用掩码：将无效位置的分数设置为-1e9（非常小的数），以在softmax中忽略它们
masked_score = score.masked_fill(mask_encoder_self_attention, -1e9)
# 计算softmax概率：沿着最后一个维度softmax，得到注意力权重
prob = F.softmax(masked_score, -1)

# Step5：构造intra-attention的mask
# Q @ K^T shape: [batch_size, tgt_seq_len, src_seq_len]
# 构建有效编码器位置：与上面类似，形状为[batch_size, max_src_seq_len, 1]，但这里用于decoder的tgt_len（注意：代码中用了max(src_len)，但逻辑上应为max_tgt_seq_len vs max_src_seq_len）
valid_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(src_len)-L)),0) for L in src_len]), 2)
# 构建有效解码器位置：对于目标序列，形状为[batch_size, max_tgt_seq_len, 1]
valid_decoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(src_len)-L)),0) for L in tgt_len]), 2)  # 注意：这里pad用了max(src_len)，但应为max(tgt_len)，可能是代码笔误
# 计算有效交叉位置矩阵：decoder_pos @ encoder_pos^T，得到[batch_size, max_tgt_seq_len, max_src_seq_len]，表示decoder位置i是否能关注encoder位置j
valid_cross_pos_matrix = torch.bmm(valid_decoder_pos, valid_encoder_pos.transpose(1, 2))
# 计算无效交叉位置矩阵：1 - valid_cross_pos_matrix
invalid_cross_pos_matrix = 1-valid_cross_pos_matrix
# 转换为布尔掩码：用于交叉注意力（encoder-decoder attention）
mask_cross_attention = invalid_cross_pos_matrix.to(torch.bool)

# Step6: 构造decoder self-attention的mask
# 构建有效解码器下三角矩阵：对于每个样本，创建下三角矩阵（tril），表示因果掩码（causal mask），只允许关注之前的位置，然后padding到max_tgt_seq_len
valid_decoder_tri_matrix = torch.cat([torch.unsqueeze(F.pad(torch.tril(torch.ones((L, L))), (0, max(tgt_len)-L, 0, max(tgt_len)-L)),0) for L in tgt_len])
# 计算无效下三角矩阵：1 - valid_decoder_tri_matrix，值为0或1
invalid_decoder_tri_matrix = 1-valid_decoder_tri_matrix
# 转换为布尔掩码：True表示无效位置（未来位置或padding）
invalid_decoder_tri_matrix = invalid_decoder_tri_matrix.to(torch.bool)

# 示例分数张量：随机生成[batch_size, max_tgt_seq_len, max_tgt_seq_len]的张量
score = torch.randn(batch_size, max(tgt_len), max(tgt_len))
# 应用掩码：将无效位置设置为-1e9
masked_score = score.masked_fill(invalid_decoder_tri_matrix, -1e9)
# 计算softmax概率：得到decoder self-attention的权重
prob = F.softmax(masked_score, -1)

# Step7: 构建scaled self-attention
# 定义缩放点积注意力函数：输入Q, K, V和注意力掩码attn_mask
def scaled_dot_product_attention(Q, K, V, attn_mask):
    # 计算分数：Q @ K^T / sqrt(d_model)，形状为[batch_size, seq_len, seq_len]（假设Q, K, V形状合适）
    score = torch.bmm(Q, K.transpose(-2, -1))/torch.sqrt(model_dim)  # 注意：torch.sqrt(model_dim)应为torch.tensor(model_dim).sqrt()以确保类型正确
    # 应用掩码：将掩码位置设置为-1e9
    masked_score = score.masked_fill(attn_mask, -1e9)
    # 计算softmax概率：沿着最后一个维度
    prob = F.softmax(masked_score, -1)
    # 计算上下文向量：prob @ V
    context = torch.bmm(prob, V)
    return context  # 返回注意力输出上下文


In [44]:
# softmax演示, scaled的重要性
alpha1 = 0.1
alpha2 = 10
score = torch.randn(5)
prob1 = F.softmax(score*alpha1, -1)
prob2 = F.softmax(score*alpha2, -1)
def softmax_func(score):
    return F.softmax(score)
jaco_mat1 = torch.autograd.functional.jacobian(softmax_func, score*alpha1)
jaco_mat2 = torch.autograd.functional.jacobian(softmax_func, score*alpha2)
print(score)
print(jaco_mat1)
print(jaco_mat2)

tensor([-0.3876,  0.1989,  0.6898,  1.0837, -0.5549])
tensor([[ 0.1527, -0.0375, -0.0394, -0.0410, -0.0348],
        [-0.0375,  0.1597, -0.0418, -0.0435, -0.0369],
        [-0.0394, -0.0418,  0.1656, -0.0457, -0.0388],
        [-0.0410, -0.0435, -0.0457,  0.1704, -0.0403],
        [-0.0348, -0.0369, -0.0388, -0.0403,  0.1508]])
tensor([[ 3.9960e-07, -5.6306e-11, -7.6300e-09, -3.9191e-07, -2.9967e-14],
        [-5.6306e-11,  1.4089e-04, -2.6905e-06, -1.3820e-04, -1.0567e-11],
        [-7.6300e-09, -2.6905e-06,  1.8730e-02, -1.8727e-02, -1.4319e-09],
        [-3.9191e-07, -1.3820e-04, -1.8727e-02,  1.8866e-02, -7.3551e-08],
        [-2.9967e-14, -1.0567e-11, -1.4319e-09, -7.3551e-08,  7.4993e-08]])


D:\anaconda3\envs\swin\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


# P_21 Transformer Masked loss 原理精讲及其PyTorch逐行实现

In [16]:
# Transformer Masked loss
import torch
import torch.nn as nn
import torch.nn.functional as F

logits = torch.randn(2, 3, 4) # batchsize=2, seqlen=3, vocab_size=4
label = torch.randint(0, 4, (2, 3))
logits = logits.transpose(1, 2)
F.cross_entropy(logits, label) # 平均交叉熵损失
F.cross_entropy(logits, label, reduction='none') # 所有的损失
tgt_len = torch.Tensor([2, 3]).to(torch.int32)
mask = torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(tgt_len)-L)), 0) for L in tgt_len])
F.cross_entropy(logits, label, reduction='none') * mask

tensor([[0.6384, 1.2946, 0.0000],
        [1.3800, 2.1678, 0.9973]])

# Transformer模型结构
![](./img/Transformer模型结构.png)

## Encoder
* input word embedding：由稀疏的one-hot向量进入一个不带bias的FNN得到一个稠密的连续向量
* position encoding
  * 通过sin/cos来固定表征
    * 每个位置确定性的
    * 对于不同的句子，相同位置的距离一直
    * 可以推广到更长的测试句子
  * pe(pos+k)可以写成pe(pos)的线性组合
  * 通过残差连接来使得位置信息流入深层
* multi-head self-attention 
  * 使得建模能力更强，表征空间更丰富
  * 由多组Q，K，V构成 每组单独计算一个attention向量
  * 把每组的attention向量拼起来，并进入一个FFN得到最终的向量
* feed-forward network
  * 只考虑每个单独位置进行建模
  * 不同位置参数共享
  * 类似于1x1 pointwise convolution

# Decoder
* output word embedding
* masked multi-head self-attention
* multi-head cross-attention
* feed-forward network
* softmax

# 总结

## 使用类型
* Encoder only： BERT、分类任务、非流式任务
* Decoder only： GPT系列、语言建模、自回归生成任务、流式任务
* Encoder-Decoder： 机器翻译、语音识别

## 特点
* 无先验假设 （例如：局部关联性、有序建模性）
* 核心计算在于自注意力机制，平方复杂度
* 数据量的要求与先验假设的程度呈反比

# seq2seq基础模块的分类

## CNN
* 权重共享
  * 平移不变性
  * 可并行计算
 * 滑动窗口 局部关联性建模 依靠多层堆积来进行长程建模
 * 对相对位置敏感，对绝对位置不敏感

## RNN
* 依次有序递归建模
  * 对顺序敏感
  * 串行计算耗时
  * 长程建模能力弱
  * 计算复杂度与序列长度呈线性关系
  * 单步计算复杂度不变
  * 对相对位置敏感，对绝对位置敏感

## transformer
* 无局部假设
  * 可并行计算
  * 对相对位置不敏感
* 无有序假设
  * 需要位置编码来反映位置变化对于特征的影响
  * 对绝对位置不敏感
* 任意两字符都可以建模
  * 擅长长短程建模
  * 自注意力机制需要序列长度的平方级别复杂度